In [1]:
import threading
import time

import json
import numpy as np
import pandas as pd
# import requests
import time
import re

from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
# from tqdm import tqdm
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")

In [86]:
executable_path = 'C:/Users/Manych/chromium/chromedriver.exe'

In [76]:
with open('./Project_7_data/marks_models_for_parsing.json') as f: 
    marks_models_for_parsing = json.loads(f.read())
# model_generation_year = pd.read_csv('./Project_7_data/model_generation_year.csv')

In [81]:
marks_models_for_parsing

dict_keys(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI', 'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'])

In [58]:

from joblib import Parallel, delayed

In [93]:
qwerty = pd.DataFrame(columns=['asd','sdf','dfg','fgh'])

def heavy(x,y):
    # имитации операций ввода-вывода
    res = pd.Series([x*y*1,x*y*2,x*y*3,x*y*4],index = ['asd','sdf','dfg','fgh'])
#     df = df.append(res,ignore_index=True)
    time.sleep(2)
    return res

In [95]:
%%time
y = 1
page_items = Parallel(n_jobs=10, prefer="threads")(
    delayed(heavy)(i,y) for i in range(1,21))
qwerty.append(page_items)

Wall time: 4.04 s


,asd,sdf,dfg,fgh
0,1,2,3,4
1,2,4,6,8
2,3,6,9,12
3,4,8,12,16
4,5,10,15,20
5,6,12,18,24
6,7,14,21,28
7,8,16,24,32
8,9,18,27,36
9,10,20,30,40


In [84]:
{k: marks_models_for_parsing[k][:2] for k in ['VOLKSWAGEN']}

{'VOLKSWAGEN': ['AMAROK', 'ARTEON']}

In [98]:


for mark in {k: marks_models_for_parsing[k][:2] for k in ['VOLKSWAGEN']}: # 
    print(mark)
    for model in marks_models_for_parsing[mark][:2]:      
        model_url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/' + '?output_type=table'
    # просмотр последовательно всех страниц текущей модели-model текущей марки-mark
        for pages_num in range(1,99): 
            if pages_num==1: page_url = model_url
            else:            page_url = model_url + '&page=' +  str(pages_num)  
            # получем html страницы
            driver = webdriver.Chrome(executable_path)
            driver.maximize_window()
            driver.get(page_url)
            page_html = driver.execute_script("return document.body.innerHTML;")
            # создаем обьект bs4.BeautifulSoup из html страницы
            page_bs = BeautifulSoup(page_html, 'html.parser') 
            # список html-ек карточек на странице                                               
            tickets_on_page = page_bs.find_all('a', class_='ListingItemTitle__link') 
            urls_on_page = [x.get('href') for x in tickets_on_page]
            print (urls_on_page)
            # выход по исчерпанию страниц текущей модели текущей марки
            if not tickets_on_page: 
                driver.quit()
                break            
#             # обработка карточек на странице
#             for ticket in tickets_on_page:
#                 # получаем url карточки текущего обьявления
#                 ticket_url = ticket.get('href')
#                 # извлекаем признаки и заполняем строку train-а
#                 train.loc[len(train)] =get_features_from_ticket(ticket_url,driver)
#                 time.sleep(0.5) 
#             time.sleep(0.5) 
            driver.quit()
#         print ('|',len(train),end=' ') # <==============================             
#     print ('|') # <============================== 

VOLKSWAGEN
['https://auto.ru/cars/used/sale/volkswagen/amarok/1114677051-498308ba/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1105973842-e70977de/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1115042580-2b8468ff/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1106543020-46ed8569/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1115036990-e02d8ef7/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1114844447-01b11af0/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1106041615-9e0b111c/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1114836194-e0068b28/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1102331043-10d93362/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1115094448-08782932/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1115038118-668f71d5/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1115190932-e1f54c77/', 'https://auto.ru/cars/used/sale/volkswagen/amarok/1114863423-de07ddd6/', 'https://auto.ru/cars/used/sale/volkswa

In [99]:
urls_on_page

[]

In [92]:
page_items = Parallel(n_jobs=10, prefer="threads")(
    delayed(get_features_from_ticket)(ticket_url,driver) for ticket_url inurls_on_page)
qwerty.append(page_items)

Wall time: 4.04 s


,asd,sdf,dfg,fgh
0,1,2,3,4
1,2,4,6,8
2,3,6,9,12
3,4,8,12,16
4,5,10,15,20
5,6,12,18,24
6,7,14,21,28
7,8,16,24,32
8,9,18,27,36
9,10,20,30,40
